In [122]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
import datetime

# Pull in GTFS data

In [134]:
df = pd.read_csv(
    '../beb_chargers/data/processed/mar_24_trip_data_complete.csv',
    dtype={'trip_id': str, 'route': str, 'vehicle_id': str},
    parse_dates=['date']
)
df = df.dropna()
df['start_time'] = pd.to_datetime(df['start_time'], utc=True).dt.tz_convert('US/Pacific')

In [135]:
trips_df = pd.read_csv(
    '../beb_chargers/data/gtfs/metro_mar24/trips.txt',
    dtype={'trip_id': str, 'shape_id': str}
)
shapes_df = pd.read_csv(
    '../beb_chargers/data/gtfs/metro_mar24/shapes.txt',
    dtype={'shape_id': str}
)

In [136]:
shape_miles = shapes_df.groupby('shape_id')['shape_dist_traveled'].max().rename('miles_static') / 5280

In [137]:
df = df.merge(
    trips_df[['trip_id', 'shape_id']], on='trip_id'
).merge(
    shape_miles, left_on='shape_id', right_index=True
)

# Predicting kwh, not kwh/mi

In [209]:
df['duration_rt']

In [212]:
df['duration_difference_pct']

In [234]:
df['60_dummy'] = df['vehicle_type'].map({'40-foot': 0, '60-foot': 1})
df['gain_per_mi'] = df['elev_gain'] / df['miles']
df['loss_per_mi'] = df['elev_loss'] / df['miles']
df['heating_degrees'] = np.maximum(65 - df['temperature'], 0)
df['heating_degree_hrs'] = df['heating_degrees'] * df['duration_rt'] / 3600
df['mph'] = 3600 * df['miles_static'] / df['duration_rt']

q1 = df['kwh_per_mi'].quantile(0.25)
q3 = df['kwh_per_mi'].quantile(0.75)
iqr = q3 - q1
ol_min = q1 - 1.5*iqr
ol_max = q3 + 1.5*iqr

df_filt = df[
    (df['kwh_per_mi'] > ol_min) & (df['kwh_per_mi'] < ol_max)
].copy()

df_filt['dayofweek'] = pd.to_datetime(df_filt['date']).dt.dayofweek
df_filt['weekday'] = df_filt['dayofweek'] <= 4
df_filt['start_hour'] = df_filt['start_time'].dt.hour
df_filt['am_peak'] = (
    (df_filt['weekday']) & (
        df_filt['start_time'].dt.hour.between(6, 8)
    )
).astype(int)
df_filt['pm_peak'] = (
    (df_filt['weekday']) & (
        df_filt['start_time'].dt.hour.between(16, 18)
    )
).astype(int)

train = df_filt[df_filt['date'] < datetime.datetime(2024, 3, 30)].copy()
test = df_filt[df_filt['date'] >= datetime.datetime(2024, 3, 30)].copy()
y = train['kwh']
X = train[['miles_static', 'heating_degree_hrs', 'gain_per_mi', 'loss_per_mi', 'duration_difference_pct', '60_dummy', 'am_peak', 'pm_peak']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()
print(results.summary())

train['prediction'] = results.predict()
train['error'] = train['prediction'] - y

In [233]:
100 * (train['error'].abs() / train['kwh']).describe()

In [227]:
train[['kwh', 'prediction', 'error']]

In [215]:
(train['error'].abs() / train['prediction']).mean()

In [230]:
train['abs_pct_error'] = 100 * train['error'].abs() / train['prediction']

In [231]:
train[['kwh', 'prediction', 'error', 'abs_pct_error']]

In [242]:
test_x = sm.add_constant(test[['miles_static', 'heating_degree_hrs', 'gain_per_mi', 'loss_per_mi', 'duration_difference_pct', '60_dummy', 'am_peak', 'pm_peak']])
test['prediction'] = results.predict(test_x)

In [243]:
ct = 0
for ix, gp in test.groupby(['date', 'vehicle_id']):
    ct += 1
    error_bar_size = 1.96*train['error'].std()

    fig = go.Figure(data=go.Scatter(
        x=gp['start_time'],
        y=gp['prediction'],
        error_y=dict(
            type='constant', # value of error bar given in data coordinates
            value=error_bar_size,
            visible=True),
        name='Prediction',
        mode='markers'
    ))

    fig.add_trace(
        go.Scatter(
            x=gp['start_time'],
            y=gp['kwh'],
            name='Observation',
            mode='markers'
        )
    )
    fig.update_layout(
        title_text=f'Bus {ix[1]} on {ix[0]}',
        yaxis_range=[test['kwh'].min() - error_bar_size, test['kwh'].max() + error_bar_size]
    )
        
    fig.show()
    if ct > 15:
        break

In [245]:
test['error'] = test['prediction'] - test['kwh']
test['abs_pct_error'] = 100 * test['error'].abs() / test['prediction']

In [246]:
test['abs_pct_error'].describe()

In [265]:
test['block_kwh'] = test.groupby(['date', 'vehicle_id'])['kwh'].cumsum()
test['block_kwh_pred'] = test.groupby(['date', 'vehicle_id'])['prediction'].cumsum()
test.sort_values(['date', 'vehicle_id', 'start_time'])[['date', 'vehicle_id', 'trip_id', 'start_time', 'block_kwh', 'block_kwh_pred']].head(20)

In [271]:
ct = 0
for ix, gp in test.groupby(['date', 'vehicle_id']):
    if len(gp) < 8:
        continue
    ct += 1
    error_bar_size = 1.96*train['error'].std()

    fig = go.Figure(data=go.Scatter(
        x=gp['start_time'],
        y=gp['block_kwh_pred'],
        # error_y=dict(
        #     type='constant', # value of error bar given in data coordinates
        #     value=error_bar_size,
        #     visible=True),
        name='Prediction'
        # mode='markers'
    ))

    fig.add_trace(
        go.Scatter(
            x=gp['start_time'],
            y=gp['block_kwh'],
            name='Observation',
            # mode='markers'
        )
    )
    fig.update_layout(
        title_text=f'Bus {ix[1]} on {ix[0]}',
        yaxis_range=[-10, max(gp['block_kwh'].max(), gp['block_kwh_pred'].max()) + error_bar_size]
    )
        
    fig.show()
    if ct > 15:
        break

In [250]:
summary_by_bus = test.groupby(['date', 'vehicle_id'])[['kwh', 'prediction']].sum()
summary_by_bus['abs_pct_error'] = 100*(summary_by_bus['prediction'] - summary_by_bus['kwh']).abs() / summary_by_bus['kwh']

In [253]:
summary_by_bus[summary_by_bus['kwh'] >= 100]['abs_pct_error'].describe()

## Hypothesized relationships
Next step for model refinement: really think these through in detail, and maybe check the literature for how others have attempted similar modeling efforts.

### Heating and cooling
The kWh of heating and cooling will be dependent on the temperature differential between indoors and outdoors, the amount of time spent heating/cooling, and the air exchange between inside and outside. It doesn't have anything to do with distance, other than the fact that distance and time are correlated.

### Elevation gain
This is all about potential energy. We need to expend energy to lift the bus up, and we retrieve energy when rolling down. There should be a linear relationship between total elevation gain/loss and total energy, since $$E_{potential} = mgh$$. However, a potential issue here is the correlation between trip distance and total gain/loss.

### Traffic conditions
If there is more traffic, buses will stop and go more often, decreasing efficiency.

### Vehicle size
A heavier bus will require more energy to accelerate, as well as to heat and cool. I think the magnitude of impact of all our variables might vary depending on whether we have a 40- or 60-foot bus.

# Refined model only

In [138]:
df['60_dummy'] = df['vehicle_type'].map({'40-foot': 0, '60-foot': 1})

In [139]:
df['gain_per_mi'] = df['elev_gain'] / df['miles']
df['loss_per_mi'] = df['elev_loss'] / df['miles']
df['heating_degrees'] = np.maximum(65 - df['temperature'], 0)
df['cooling_degrees'] = np.maximum(df['temperature'] - 65, 0)

In [140]:
df['mph'] = 3600 * df['miles_static'] / df['duration_rt']

In [148]:
q1 = df['kwh_per_mi'].quantile(0.25)
q3 = df['kwh_per_mi'].quantile(0.75)
iqr = q3 - q1
ol_min = q1 - 1.5*iqr
ol_max = q3 + 1.5*iqr

df_filt = df[
    (df['kwh_per_mi'] > ol_min) & (df['kwh_per_mi'] < ol_max)
].copy()

df_filt['dayofweek'] = pd.to_datetime(df_filt['date']).dt.dayofweek
df_filt['weekday'] = df_filt['dayofweek'] <= 4
df_filt['start_hour'] = df_filt['start_time'].dt.hour
df_filt['am_peak'] = (
    (df_filt['weekday']) & (
        df_filt['start_time'].dt.hour.between(6, 8)
    )
).astype(int)
df_filt['pm_peak'] = (
    (df_filt['weekday']) & (
        df_filt['start_time'].dt.hour.between(16, 18)
    )
).astype(int)

train = df_filt[df_filt['date'] < datetime.datetime(2024, 3, 30)].copy()
test = df_filt[df_filt['date'] >= datetime.datetime(2024, 3, 30)].copy()
y = train['kwh_per_mi']
X = train[['heating_degrees', 'gain_per_mi', 'loss_per_mi', 'mph', '60_dummy', 'am_peak', 'pm_peak']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()
print(results.summary())

train['prediction'] = results.predict()
train['error'] = train['prediction'] - train['kwh_per_mi']

In [149]:
test_x = sm.add_constant(test[['heating_degrees', 'gain_per_mi', 'loss_per_mi', 'mph', '60_dummy', 'am_peak', 'pm_peak']])
test['prediction'] = results.predict(test_x)

In [171]:
ct = 0
for ix, gp in test.groupby(['date', 'vehicle_id']):
    ct += 1

    fig = go.Figure(data=go.Scatter(
        x=gp['start_time'],
        y=gp['prediction'],
        error_y=dict(
            type='constant', # value of error bar given in data coordinates
            value=2*train['error'].std(),
            visible=True),
        name='Prediction',
        mode='markers'
    ))

    fig.add_trace(
        go.Scatter(
            x=gp['start_time'],
            y=gp['kwh_per_mi'],
            name='Observation',
            mode='markers'
        )
    )
    fig.update_layout(
        title_text=f'Bus {ix[1]} on {ix[0]}',
        yaxis_range=[test['kwh_per_mi'].min(), test['kwh_per_mi'].max()]
    )
    fig.write_image(f'images/predictions_{ct}.pdf')

In [157]:
test['kwh'] = test['kwh_per_mi'] * test['miles_static']

In [158]:
test['kwh_predicted'] = test['prediction'] * test['miles_static']

In [193]:
test['kwh_var'] = test['miles_static'] * train['error'].var()

In [200]:
train['error'].std()

In [187]:
test['kwh_lower_cl'] = test['miles_static'] * (test['prediction'] - 1.96*train['error'].std())
test['kwh_upper_cl'] = test['miles_static'] * (test['prediction'] + 1.96*train['error'].std())

In [204]:
test[['vehicle_id', 'trip_id', 'kwh', 'kwh_predicted', 'kwh_std', 'kwh_var', 'kwh_lower_cl', 'kwh_upper_cl']]

In [190]:
bus_totals = test.groupby(['date', 'vehicle_id'])[['kwh', 'kwh_predicted']].sum()

In [196]:
bus_totals['kwh_std'] = np.sqrt(test.groupby(['date', 'vehicle_id'])['kwh_var'].sum())

In [197]:
bus_totals

In [ ]:

bus_totals['n_trips'] = test.groupby(['date', 'vehicle_id'])['trip_id'].count()
bus_totals = bus_totals.reset_index()

In [174]:

px.scatter(
    bus_totals,
    x='kwh_predicted',
    y='kwh',
    color='vehicle_id'
)

In [177]:
bus_totals['std'] = np.sqrt(bus_totals['n_trips'] * train['error'].std())

In [167]:
bus_totals['pct_error'] = 100 * (bus_totals['kwh_predicted'] - bus_totals['kwh']) / bus_totals['kwh']

In [178]:
bus_totals

In [170]:
px.scatter(
    bus_totals,
    x='kwh',
    y='pct_error',
    color='date'
)

In [ ]:
bus_totals 

## Try to plot prediction accuracy

In [154]:
ct = 0
for ix, gp in df_filt.groupby(['date', 'vehicle_id']):
    ct += 1

    fig = go.Figure(data=go.Scatter(
        x=gp['start_time'],
        y=gp['prediction'],
        error_y=dict(
            type='constant', # value of error bar given in data coordinates
            value=1.96*df_filt['error'].std(),
            visible=True),
        name='Prediction',
        mode='markers'
    ))

    fig.add_trace(
        go.Scatter(
            x=gp['start_time'],
            y=gp['kwh_per_mi'],
            name='Observation',
            mode='markers'
        )
    )
    fig.update_layout(
        title_text=f'Bus {ix[1]} on {ix[0]}',
        yaxis_range=[test['kwh_per_mi'].min(), df_filt['kwh_per_mi'].max()]
    )
        
    fig.show()
    if ct > 15:
        break

In [49]:
df_filt.groupby(['date', 'vehicle_id'])['error'].sum().hist()

In [60]:
px.scatter(
    df,
    x='temperature',
    y='kwh_per_mi',
    color='vehicle_type'
)

In [62]:
go.Figure(
    go.Heatmap(
        x=df['temperature'],
        y=df['pct_grade'],
        z=df['kwh_per_mi'],
        # color='vehicle_type'
    )
).show()

In [44]:
df_filt[
    (df_filt['vehicle_id'] == 4705)
    & (df_filt['date'] == '2024-03-02')
]

In [76]:
import matplotlib.pyplot as plt

In [78]:
df_filt['error'].hist(bins=30)
plt.xlabel('Error')
plt.ylabel('Count')
plt.title('Residuals')

In [82]:
df_filt.groupby(['route', 'direction_id'])['error'].median().sort_values()

# Archive of modeling attempts

In [4]:
y = df['kwh_per_mi']
X = df[['pct_grade', '60_dummy']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()

In [5]:
print(results.summary())

In [6]:
df['prediction'] = results.predict()

In [7]:
fig = px.scatter(df, 'pct_grade', 'prediction', color='vehicle_type')
fig.add_trace(
    go.Scatter(
        x=df['pct_grade'], y=df['kwh_per_mi'], fillcolor='gray', mode='markers', marker_color='gray', name='Observed data'
    )
)
fig.update_layout(
    title='Energy Consumption Versus Elevation: Predictions',
    xaxis_title='Trip Slope (% grade)',
    yaxis_title='Energy Consumption (kWh/mile)',
    legend_title=None
)
# fig.write_image('predictions.pdf')
fig.show()

In [8]:
df['dayofweek'] = pd.to_datetime(df.date).dt.dayofweek

In [9]:
df['tuesday'] = df['dayofweek'] == 1
df['wednesday'] = df['dayofweek'] == 2
df['thursday'] = df['dayofweek'] == 3
df['friday'] = df['dayofweek'] == 4
df['saturday'] = df['dayofweek'] == 5
df['sunday'] = df['dayofweek'] == 6
df[['tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']] = \
    df[['tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']].astype(int)

In [10]:
df['mph'] = 3600 * df['miles'] / df['duration_rt']
y = df['kwh_per_mi']
X = df[['pct_grade', '60_dummy', 'mph', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()

In [11]:
print(results.summary())

In [12]:
df['weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
y = df['kwh_per_mi']
X = df[['pct_grade', '60_dummy', 'mph', 'weekend']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()

In [13]:
print(results.summary())

# What if we model gain and loss per mile separately?

In [14]:
df['gain_per_mi'] = df['elev_gain'] / df['miles']
df['loss_per_mi'] = df['elev_loss'] / df['miles']

In [15]:
y = df['kwh_per_mi']
X = df[['gain_per_mi', 'loss_per_mi', '60_dummy', 'mph', 'weekend']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()

In [16]:
print(results.summary())

In [17]:
df['prediction'] = results.predict()
df['error'] = df['prediction'] - df['kwh_per_mi']

In [18]:
df['error'].hist()

In [19]:
error_summary = df[['vehicle_id', 'vehicle_type', 'date', 'start_time', 'trip_id', 'route', 'mph', 'kwh_per_mi', 'prediction', 'error']]

In [20]:
error_summary[error_summary['error'] > 0.7].sort_values('date').head(20)

In [21]:
error_summary.groupby('vehicle_type')['error'].hist(legend=True)

In [23]:
import numpy as np

In [24]:
df['heating_degrees'] = np.maximum(65 - df['temperature'], 0)
df['cooling_degrees'] = np.maximum(df['temperature'] - 65, 0)

In [25]:
# Throw out trips with less than 10 mph
df = df[df['mph'] > 10].copy()

In [26]:
y = df['kwh_per_mi']
X = df[['heating_degrees', 'gain_per_mi', 'loss_per_mi', 'mph', '60_dummy', 'weekend']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()

In [27]:
print(results.summary())

In [28]:
df['prediction'] = results.predict()
df['error'] = df['prediction'] - df['kwh_per_mi']

In [29]:
df['error'].max()

In [30]:
df[df['error'] == df['error'].max()][['heating_degrees', 'gain_per_mi', 'loss_per_mi', 'miles', '60_dummy', 'mph', 'weekend', 'kwh_per_mi', 'prediction']]

In [31]:
import matplotlib.pyplot as plt

In [32]:
(100 * df['error'] / df['kwh_per_mi']).hist(bins=30)
plt.xlabel('Percent Error')
plt.ylabel('Count')
plt.title('Distribution of Model Errors')
plt.show()

In [33]:
df['error'].std()

In [34]:
df_40 = df[df['vehicle_type'] == '40-foot'].copy()
fig = px.scatter(df_40, 'heating_degrees', 'kwh_per_mi', color='route')
fig.update_layout(
    title='Energy Consumption Versus Temperature: 40-Foot Buses',
    xaxis_title='Temperature (degrees F)',
    yaxis_title='Energy Consumption (kWh/mile)'
)
# fig.write_image('elevation_impact_40ft.pdf')
fig.show()

In [35]:
df_40.groupby('temperature')['kwh_per_mi'].mean().sort_index().plot()

## Trying one more possible improvement: peak/off-peak periods

In [39]:
df['start_time'] = pd.to_datetime(df['start_time'], utc=True).dt.tz_convert('US/Pacific')
df['dayofweek'] = pd.to_datetime(df['date']).dt.dayofweek
df['weekday'] = df['dayofweek'] <= 4
df['start_hour'] = df['start_time'].dt.hour
df['am_peak'] = (
    (df['weekday']) & (
        df['start_time'].dt.hour.between(5, 8)
    )
).astype(int)
df['pm_peak'] = (
    (df['weekday']) & (
        df['start_time'].dt.hour.between(16, 20)
    )
).astype(int)

In [40]:
y = df['kwh_per_mi']
X = df[['heating_degrees', 'gain_per_mi', 'loss_per_mi', 'mph', '60_dummy', 'pm_peak']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()

In [41]:
print(results.summary())

In [42]:
df['prediction'] = results.predict()
df['error'] = df['prediction'] - df['kwh_per_mi']

In [43]:
(100 * df['error'] / df['kwh_per_mi']).hist(bins=30)
plt.xlabel('Percent Error')
plt.ylabel('Count')
plt.title('Distribution of Model Errors')
plt.show()

In [44]:
df['pct_error'] = 100 * df['error'] / df['kwh_per_mi']
df['pct_error'].max()

In [45]:
df[df['pct_error'] == df['pct_error'].max()][
    ['kwh_per_mi', 'prediction', 'error', 'pct_error']
]

## Try removing outliers and fitting again

In [46]:
q1 = df['kwh_per_mi'].quantile(0.25)
q3 = df['kwh_per_mi'].quantile(0.75)
iqr = q3 - q1
ol_min = q1 - 1.5*iqr
ol_max = q3 + 1.5*iqr

In [47]:
df_filt = df[
    (df['kwh_per_mi'] > ol_min) & (df['kwh_per_mi'] < ol_max)
].copy()

In [48]:
len(df), len(df_filt)

In [49]:
df_filt['start_time'] = pd.to_datetime(df_filt['start_time'])
df_filt['dayofweek'] = pd.to_datetime(df_filt['date']).dt.dayofweek
df_filt['weekday'] = df_filt['dayofweek'] <= 4
df_filt['start_hour'] = df_filt['start_time'].dt.hour
df_filt['am_peak'] = (
    (df_filt['weekday']) & (
        df_filt['start_time'].dt.hour.between(5, 8)
    )
).astype(int)
df_filt['pm_peak'] = (
    (df_filt['weekday']) & (
        df_filt['start_time'].dt.hour.between(16, 20)
    )
).astype(int)

In [56]:
y = df_filt['kwh_per_mi']
X = df_filt[['heating_degrees', 'gain_per_mi', 'loss_per_mi', 'mph', '60_dummy', 'am_peak', 'pm_peak']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
results = mod.fit()
print(results.summary())

In [57]:
df_filt['prediction'] = results.predict()
df_filt['error'] = df_filt['prediction'] - df_filt['kwh_per_mi']

In [58]:
df_filt['error'].hist()

In [59]:
df_filt['error'].describe()